# Investigating Variable Renaming
The goal of this notebook is to understand the mechanism by which a transformer
learns to evaluate variable renamings across various depths.

### Analysis
1) Staring at attention weights
2) Patching at various depths
3) Examining embeddings

- Maybe do some SVD or PCA to visualize embeddings?

### Hypotheses
- I think the model mostly duplicates information that must be retrieved later
- The model likely guesses behind using different heads
- The model probably just attends to a bunch at the same time
- Input embeddings are likely very orthogonal

In [1]:
import torch
import numpy as np

from iluvattnshun.utils import load_checkpoint, load_config_from_yaml
from iluvattnshun.nn import MultilayerTransformer
from iluvattnshun.viz import get_fig

from var_rename import VariableRenamingConfig, VariableRenamingPrompter

/home/michael-lutz/miniconda3/envs/ila/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-05 20:24:22.696231: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-05 20:24:22.711048: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751747062.728880  986609 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751747062.734191  986609 cuda_blas.

In [ ]:
config_path = "/home/michael-lutz/iluvattnshun/logs/var_rename/run_44_sweep_7/run_44_sweep_7.yaml"
ckpt_path = "/home/michael-lutz/iluvattnshun/logs/var_rename/run_44_sweep_7/ckpt_epoch_167.pt"

config = load_config_from_yaml(config_path, VariableRenamingConfig)

# loading the model
max_seq_len = config.num_renames * 4
model = MultilayerTransformer(
    vocab_size=39,
    d_model=config.dim_model,
    n_heads=config.num_heads,
    n_layers=config.num_layers,
)
load_checkpoint(ckpt_path, model)
model.eval()

# loading the prompter
prompter = VariableRenamingPrompter(config)

FileNotFoundError: [Errno 2] No such file or directory: '/home/michael-lutz/iluvattnshun/logs/var_rename/run_44_sweep_7/ckpt_epoch_166.pt'

In [ ]:
rng = np.random.default_rng(42)
prompt, answer, metadata = prompter.get_prompt(rng)
x = torch.tensor(prompter.tokenize(prompt)).unsqueeze(0)
logits, _, attn_weights, _= model.forward(x, return_attn_weights=True, return_xs=True)
pred = prompter.detokenize(logits[0].argmax(dim=-1).tolist())

print("prompt: ", prompt)
print("pred:   ", pred)
print("answer: ", answer)

prompt:  2>l;0>c;1>f;3>p;c>u;u>s;s>n;p>w;l>o;o>e;n>u;f>k;u>n;e>l;w>c;k>x;c>v;n>g;x>d;g>s;l>b;s>l;l>r;r>t;v>k;b>n;k>p;p>u;d>y;y>i;t>k;n>x;x>b;b>v;u>m;m>s;v>h;s>p;i>y;h>d;k>q;y>c;d>u;q>k;k>w;u>x;x>g;c>r;p>v;v>t;t>u;w>t;r>l;l>h;t>o;g>n;h>a;u>g;g>l;a>s;n>w;s>b;o>p;b>o;o>b;b>u;w>o;l>h;o>z;h>g;z>y;p>a;g>v;v>t;y>x;x>k;u>d;d>o;a>z;o>j;k>l;z>h;t>i;i>c;c>u;j>s;l>r;u>x;s>y;x>l;y>m;r>d;d>g;g>t;m>q;t>z;l>i;i>j;h>k;k>m;q>l;z>t;m>f;f>g;l>v;t>s;s>b;j>m;g>d;b>s;m>r;s>u;v>n;r>l;d>a;a>m;n>r;u>g;r>d;d>c;c>r;l>u;g>v;u>d;v>f;f>s;d>q;q>d;r>v;d>h;m>z;v>k;s>p;k>g;h>d;z>l;p>n;l>b;d>j;n>s;s>m;b>e;g>p;m>v;v>y;y>n;p>w;n>h;j>l;e>a;
pred:    222200001111333100000000000033302220222200001111000022223333111133330000111100002222000000000000333322223333333311111111000022202020202033303330222233331111222200001110222200000000222222221111333031333333000011101111000022221111333333331111222211110000111111101110222033302221333322230000333331332222222211111111000011112222000033303233333311112221333311123333111122212222222211112

Worth nothing that 

In [ ]:
weights = [attn_weight.detach().cpu().numpy()[0] for attn_weight in attn_weights]
plotly_fig = get_fig(weights, list(prompt), selected_layers=[0], selected_heads=[0, 1], selected_x_toks=list(range(60)), selected_y_toks=list(range(60)))
plotly_fig.show()

In [ ]:
weights = [attn_weight.detach().cpu().numpy()[0] for attn_weight in attn_weights]
plotly_fig = get_fig(weights, list(prompt), selected_layers=[0], selected_heads=[0, 1], selected_x_toks=list(range(0,601)), selected_y_toks=list(range(560,601)))
plotly_fig.show()